# Iherb Crawler

In [ ]:
pip install selenium webdriver-manager ipykernel

In [ ]:
# 필요한 라이브러리 로드
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import json

In [ ]:
# Exception 클래스를 상속받은 InputError를 설명하는 클래스 생성 > url이 입력 되지 않았을경우 raise
class InputError(Exception):
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """
    def __init__(self, expression=None, message=None):
        if not expression or not message:
            print('empty url: URL을 전달해주세요.')

In [ ]:
class IherbCrawler():
    def __init__(self, url: str):
        if len(url) == 0:
            raise InputError()
        self.url = url


    def make_pagenation(self, start_nums: int, last_nums: int):
        '''
        ## url의 {page_nums}로 된 부분을 숫자 형식에 맞게 치환하는 함수\n
        args:\n
        start_nums: 페이지네이션 된 페이지의 시작 페이지\n
        last_nums: 원하는 페이지의 갯수\n
        return: pagination된 url list
        '''
        if start_nums <= 0 or last_nums <= 0:
            raise ValueError
        
        url_list = []
        for i in range(start_nums, start_nums+last_nums):
            url = self.url.replace('page_nums', str(i))
            url_list.append(url)

        return url_list
    
    # driver를 연 후 페이지네이션처리하여 접근하면 cloudflare로 인해 접근이 막힌다. 따라서 page별 driver를 재생성하여 데이터에 접근한다
    def get_crawling_data(self, url_list: list, class_dict: dict):
        '''
        ### url_list를 전달 받고 이를 class_dict에 작성 되어 있는 tag의 class_name을 찾아 데이터를 처리 후 리스트로 반환하는 함수
        '''
        data_list = []
        rank = 1
        for url in url_list:
            with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as crawler:
                page_data = []
                crawler.get(url=url)
                crawler.implicitly_wait(5)
                display_name_list = crawler.find_elements(By.CLASS_NAME, class_dict.get("tag_class_name1"))
                original_price_list = crawler.find_elements(By.CLASS_NAME, class_dict.get("tag_class_name2"))
                original_price_list.pop(0)
                image_url_list = crawler.find_elements(By.CLASS_NAME, class_dict.get("img_class_name"))

                # 추출된 데이터를 json 형식에 맞게 치환하여 data_list에 담는 작업
                num = rank
                for i in range(len(image_url_list)):
                    data = {"shop_name": "iHerb"}
                    data.update({"display_name": ''.join(display_name_list[i].text.split(',')[1:])[1:]})
                    data.update({"brand_name": display_name_list[i].text.split(',')[0]})
                    data.update({"original_price": int(original_price_list[i].text.replace("₩", "").replace(",", ""))})
                    data.update({"sale_price": int(original_price_list[i].text.replace("₩", "").replace(",", ""))})
                    data.update({"image_url": image_url_list[i].find_element(By.TAG_NAME, "img").get_attribute("src")})
                    data.update({"rank": num+i})
                    page_data.append(data)
                    rank = num+i

                data_list.append(page_data) 

        return data_list

In [ ]:
# page_nums로 페이지네이션 부분을 치환하여 url로 저장한다
url = "https://kr.iherb.com/c/supplements?_gl=1*rdcxv1*_up*MQ..&gclid=CjwKCAjwuePGBhBZEiwAIGCVS72VuNgVm_QhdDVM8zmaM3LQeXlCXC6gyYOYVj6nOOZJhI4bSlNUbhoC7mkQAvD_BwE&gclsrc=aw.ds&soa=true&sr=2&p=page_nums"


crawler = IherbCrawler(url=url)
urls = crawler.make_pagenation(1, 3)

# 데이터 추출을 원하는 tag의 class name을 딕셔너리 형식으로 작성한다.
class_dict = {"tag_class_name1": "product-title", "tag_class_name2": "price ", "img_class_name": "product-image"}
crawling_data = crawler.get_crawling_data(urls, class_dict)

In [ ]:
# 추출한 데이터를 토대로 .json 파일로 저장함
with open("data.json", 'r+', encoding="utf-8") as f:
    f.write("[")  # JSON 배열 시작
    for page_data in crawling_data:
        for i, data in enumerate(page_data):
            json.dump(data, f, ensure_ascii=False, indent=4)
            f.write(",")
    pos = f.tell() - 1
    f.truncate(pos)
    f.write("]")  # JSON 배열 끝